# Predicción del crecimiento de ganancias empresariales en Colombia

Este notebook documenta de forma reproducible el proceso completo de **limpieza, análisis exploratorio y modelado predictivo** aplicado al conjunto de datos de las *10.000 empresas más grandes de Colombia* (para varios años).

El objetivo práctico es construir un modelo capaz de **estimar las ganancias futuras de las empresas** a partir de su información histórica, de manera que sirva como base para:

- Analizar el **potencial de rentabilidad por sector y región**.
- Identificar patrones de crecimiento en diferentes grupos de empresas.
- Proveer insumos cuantitativos para un **dashboard interactivo** enfocado en decisiones de política pública, inversión y desarrollo empresarial.

A lo largo del notebook se cubren las siguientes etapas:

1. Carga y comprensión básica del dataset.
2. Limpieza de datos e identificación robusta de empresas.
3. Construcción de variables financieras derivadas (ratios).
4. Análisis exploratorio descriptivo por año y por macrosector.
5. Construcción de un panel temporal empresa–año y del target de predicción.
6. Entrenamiento y evaluación de varios modelos predictivos:
   - Baseline de persistencia (ganancia futura = ganancia actual).
   - Modelos de regresión basados en árboles (Random Forest).
   - Un enfoque basado en el cambio de ganancia (**Δganancia**).
   - Un esquema de **dos etapas** (clasificación + regresión).
7. Selección del modelo final y análisis de importancia de variables.

El foco está en justificar las decisiones de modelado y mostrar con claridad **qué modelo funciona mejor y por qué**, más allá de “probar modelos a ciegas”.


In [25]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    accuracy_score,
    roc_auc_score,
)
from sklearn.inspection import permutation_importance

pd.set_option("display.max_columns", 100)
sns.set(style="whitegrid")

RANDOM_STATE = 42

## 1. Carga de datos y exploración inicial

En esta sección se carga el archivo de las empresas más grandes, se hace una inspección básica de su estructura y se normalizan algunos nombres de columnas para facilitar el trabajo posterior.

In [26]:
# Ruta al archivo de datos (ajústala si es necesario)
DATA_PATH = "greatest_colombian_business.csv"

df_raw = pd.read_csv("../data/greatest_colombian_business.csv")
print(df_raw.shape)
df_raw.head()

(40000, 14)


,NIT,RAZÓN SOCIAL,SUPERVISOR,REGIÓN,DEPARTAMENTO DOMICILIO,CIUDAD DOMICILIO,CIIU,MACROSECTOR,INGRESOS OPERACIONALES,GANANCIA (PÉRDIDA),TOTAL ACTIVOS,TOTAL PASIVOS,TOTAL PATRIMONIO,Año de Corte
0,"899,999,068",ECOPETROL S.A,SUPERFINANCIERA,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,610,MINERO,$144.82,$33.41,$216.85,$125.81,$91.03,"2,022"
1,"900,112,515",REFINERIA DE CARTAGENA S.A.S,SUPERSOCIEDADES,Costa Atlántica,BOLIVAR,CARTAGENA-BOLIVAR,"1,921",MANUFACTURA,$27.86,$2.19,$42.84,$16.48,$26.36,"2,022"
2,"830,095,213",ORGANIZACIÓN TERPEL S.A.,SUPERFINANCIERA,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,"4,661",COMERCIO,$23.60,$0.33,$7.48,$4.47,$3.01,"2,022"
3,"860,069,804",CARBONES DEL CERREJON LIMITED,SUPERSOCIEDADES,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,510,MINERO,$16.39,$6.05,$10.45,$9.00,$1.45,"2,022"
4,"800,021,308",DRUMMOND LTD,SUPERSOCIEDADES,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,510,MINERO,$15.27,$2.16,$14.27,$6.34,$7.93,"2,022"


A partir de la vista inicial se observa que el dataset contiene información para varias columnas financieras (ingresos, ganancia, activos, pasivos, patrimonio), así como variables de identificación de la empresa (NIT, razón social), sector económico, región y año de corte.

Antes de realizar cualquier análisis cuantitativo es necesario homogeneizar los nombres de las columnas y asegurarnos de que los campos numéricos estén efectivamente en formato numérico.


In [27]:
df = df_raw.copy()

# Normalizar nombres de columnas: quitar espacios y acentos básicos
df.columns = (
    df.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.upper()
    .str.replace("Ó", "O")
    .str.replace("Í", "I")
    .str.replace("Á", "A")
    .str.replace("É", "E")
    .str.replace("Ú", "U")
    .str.replace("Ñ", "N")
)

print(df.columns)
df.head(3)

Index(['NIT', 'RAZON_SOCIAL', 'SUPERVISOR', 'REGION', 'DEPARTAMENTO_DOMICILIO',
       'CIUDAD_DOMICILIO', 'CIIU', 'MACROSECTOR', 'INGRESOS_OPERACIONALES',
       'GANANCIA_(PERDIDA)', 'TOTAL_ACTIVOS', 'TOTAL_PASIVOS',
       'TOTAL_PATRIMONIO', 'ANO_DE_CORTE'],
      dtype='object')


,NIT,RAZON_SOCIAL,SUPERVISOR,REGION,DEPARTAMENTO_DOMICILIO,CIUDAD_DOMICILIO,CIIU,MACROSECTOR,INGRESOS_OPERACIONALES,GANANCIA_(PERDIDA),TOTAL_ACTIVOS,TOTAL_PASIVOS,TOTAL_PATRIMONIO,ANO_DE_CORTE
0,"899,999,068",ECOPETROL S.A,SUPERFINANCIERA,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,610,MINERO,$144.82,$33.41,$216.85,$125.81,$91.03,"2,022"
1,"900,112,515",REFINERIA DE CARTAGENA S.A.S,SUPERSOCIEDADES,Costa Atlántica,BOLIVAR,CARTAGENA-BOLIVAR,"1,921",MANUFACTURA,$27.86,$2.19,$42.84,$16.48,$26.36,"2,022"
2,"830,095,213",ORGANIZACIÓN TERPEL S.A.,SUPERFINANCIERA,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,"4,661",COMERCIO,$23.60,$0.33,$7.48,$4.47,$3.01,"2,022"


## 2. Limpieza de identificadores y variables monetarias

En esta sección se abordan tres problemas centrales de calidad de datos:

1. Asegurar que el **NIT** se comporte como un identificador estable de empresa.
2. Normalizar el campo **CIUDAD_DOMICILIO**, eliminando inconsistencias simples de formato.
3. Convertir a tipo numérico todas las variables financieras que vienen como texto con símbolos (`$`, comas, etc.).

Estas decisiones son críticas porque cualquier error aquí se propaga a todo el análisis posterior.


In [28]:
# Limpiar NIT: quitar comas, espacios y forzar a string
df["NIT"] = (
    df["NIT"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.strip()
)

# Normalizar ciudad: quitar comas, espacios repetidos y espacios alrededor de "-"
df["CIUDAD_DOMICILIO"] = (
    df["CIUDAD_DOMICILIO"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .str.replace(r"\s*-\s*", "-", regex=True)
)

def clean_ciudad(ciudad: str) -> str:
    """
    Si la ciudad viene con repeticiones del departamento, por ejemplo
    'ITAGUI-ANTIOQUIA-ANTIOQUIA', se elimina el último segmento repetido.
    """
    parts = str(ciudad).split("-")
    if len(parts) >= 3 and parts[-1] == parts[-2]:
        parts = parts[:-1]
    return "-".join(parts)

df["CIUDAD_DOMICILIO"] = df["CIUDAD_DOMICILIO"].apply(clean_ciudad)

df[["NIT", "CIUDAD_DOMICILIO"]].head()

,NIT,CIUDAD_DOMICILIO
0,899999068,BOGOTA D.C.-BOGOTA D.C.
1,900112515,CARTAGENA-BOLIVAR
2,830095213,BOGOTA D.C.-BOGOTA D.C.
3,860069804,BOGOTA D.C.-BOGOTA D.C.
4,800021308,BOGOTA D.C.-BOGOTA D.C.


La limpieza de `NIT` y `CIUDAD_DOMICILIO` permite tratar a estas variables como claves y atributos confiables.

En particular:

- El **NIT** se usa como clave primaria para construir el panel empresa–año.
- La combinación `(NIT, CIUDAD_DOMICILIO)` ayuda a evitar que empresas con nombres similares en ciudades distintas se mezclen por error en etapas de limpieza de texto.


### 2.1 Conversión de año y columnas monetarias

El campo de año puede venir con formatos como `"2,021"`, por lo que se limpia y convierte a entero. Las columnas financieras (`INGRESOS_OPERACIONALES`, `GANANCIA_(PERDIDA)`, `TOTAL_ACTIVOS`, `TOTAL_PASIVOS`, `TOTAL_PATRIMONIO`) se convierten a `float` después de eliminar símbolos y separadores.


In [29]:
# Limpiar año de corte: por ejemplo "2,021" -> 2021 como entero
df["ANO_DE_CORTE"] = (
    df["ANO_DE_CORTE"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .astype(int)
)

money_cols = [
    "INGRESOS_OPERACIONALES",
    "GANANCIA_(PERDIDA)",
    "TOTAL_ACTIVOS",
    "TOTAL_PASIVOS",
    "TOTAL_PATRIMONIO",
]

def clean_money(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.replace("$", "", regex=False)
         .str.replace(",", "", regex=False)
         .str.strip()
         .replace("", np.nan)
         .astype(float)
    )

for col in money_cols:
    df[col] = clean_money(df[col])

df[money_cols].describe()

,INGRESOS_OPERACIONALES,GANANCIA_(PERDIDA),TOTAL_ACTIVOS,TOTAL_PASIVOS,TOTAL_PATRIMONIO
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,0.167670,0.013160,0.218061,0.110702,0.107080
std,1.392991,0.254042,2.369618,1.345162,1.117528
min,0.010000,-3.210000,0.000000,0.000000,-3.690000
25%,0.030000,0.000000,0.020000,0.010000,0.010000
50%,0.040000,0.000000,0.030000,0.020000,0.010000
75%,0.100000,0.010000,0.090000,0.050000,0.040000
max,144.820000,33.410000,216.850000,130.540000,91.030000


Las estadísticas descriptivas de las variables monetarias muestran inmediatamente varias características del dataset:

- Las distribuciones de **ingresos, activos, pasivos y patrimonio** son fuertemente asimétricas: la media es muy superior a la mediana y existen valores máximos muy altos, típicos de unas pocas empresas extremadamente grandes.
- La variable **ganancia** (`GANANCIA_(PERDIDA)`) presenta una mediana igual a cero, lo que indica que más de la mitad de las empresas reportan ganancias cercanas a cero o exactamente cero.
- Existen valores negativos en `GANANCIA_(PERDIDA)` y en `TOTAL_PATRIMONIO`, lo que refleja la presencia de empresas con pérdidas y patrimonio negativo.

Esta estructura —muchas observaciones cercanas a cero y pocas observaciones muy grandes— es típica en datos empresariales y tendrá impacto directo en la elección de métricas de evaluación y en las transformaciones aplicadas al modelado (por ejemplo, el uso de transformaciones logarítmicas).

## 3. Calidad de la razón social e identificación de empresas

La **razón social** es relevante principalmente a nivel descriptivo y para presentación de resultados. Sin embargo, es importante asegurarse de que no haya huecos en esta columna y que sea consistente para cada combinación `(NIT, CIUDAD_DOMICILIO)`.

En esta sección se:

1. Revisa el número de valores faltantes en `RAZON_SOCIAL`.
2. Imputa las razones sociales faltantes usando el valor más frecuente para cada pareja `(NIT, CIUDAD_DOMICILIO)`.
3. Verifica la consistencia básica entre NIT y razón social.


In [30]:
df.isna().sum()

NIT                       0
RAZON_SOCIAL              2
SUPERVISOR                0
REGION                    0
DEPARTAMENTO_DOMICILIO    0
CIUDAD_DOMICILIO          0
CIIU                      0
MACROSECTOR               0
INGRESOS_OPERACIONALES    0
GANANCIA_(PERDIDA)        0
TOTAL_ACTIVOS             0
TOTAL_PASIVOS             0
TOTAL_PATRIMONIO          0
ANO_DE_CORTE              0
dtype: int64

In [31]:
# Imputar razón social faltante usando el modo por (NIT, CIUDAD_DOMICILIO)
def modo_serie(s: pd.Series):
    m = s.mode()
    if len(m) == 0:
        return np.nan
    return m.iloc[0]

canon_rs = (
    df.dropna(subset=["RAZON_SOCIAL"])
      .groupby(["NIT", "CIUDAD_DOMICILIO"])["RAZON_SOCIAL"]
      .agg(modo_serie)
      .reset_index()
      .rename(columns={"RAZON_SOCIAL": "RAZON_SOCIAL_CANONICA"})
)

df = df.merge(canon_rs, on=["NIT", "CIUDAD_DOMICILIO"], how="left")
df["RAZON_SOCIAL"] = df["RAZON_SOCIAL_CANONICA"]
df.drop(columns=["RAZON_SOCIAL_CANONICA"], inplace=True)

df["RAZON_SOCIAL"].isna().sum()


np.int64(0)

En caso de existir muy pocos registros sin razón social incluso después de este proceso, pueden eliminarse sin afectar las conclusiones, dada la escala del dataset.

El punto clave es que, a partir de este momento, cada empresa se identifica de manera robusta por su `NIT`, y los atributos de nombre y ciudad asociado son coherentes para el análisis descriptivo y para el posterior consumo en un dashboard.


## 4. Estructura temporal y construcción de ratios financieros

Antes de modelar es fundamental entender:

1. **Cuántos años de información hay por empresa**, lo que define qué tan rico es el panel para predicción temporal.
2. La distribución de variables clave por año (ingresos, ganancias).
3. Algunos **ratios financieros** básicos que capturan rentabilidad y estructura de capital:
   - Margen de ganancia (`GANANCIA / INGRESOS`).
   - Retorno sobre activos (ROA).
   - Retorno sobre patrimonio (ROE).
   - Apalancamiento (`PASIVOS / ACTIVOS`).

Estos ratios suelen ser más interpretables para análisis sectoriales y sirven como insumo natural para el modelo predictivo.


In [32]:
# Distribución de cantidad de años por empresa (NIT)
years_per_nit = (
    df.groupby("NIT")["ANO_DE_CORTE"]
      .nunique()
      .value_counts()
      .sort_index()
)
years_per_nit


ANO_DE_CORTE
1    2950
2    2584
3    1886
4    6556
Name: count, dtype: int64

La distribución de años por NIT muestra cuántas empresas tienen 1, 2, 3 o 4 observaciones en el panel. Un número significativo de empresas con cuatro años de información es una buena noticia: permite construir pares de entrenamiento del tipo *año t → año t+1* de manera robusta.

A continuación se construyen los ratios financieros más importantes. Las divisiones se realizan de forma segura, evitando denominadores cero o negativos cuando no tienen sentido económico.


In [33]:
# Construcción de ratios financieros
df["margen"] = np.where(
    df["INGRESOS_OPERACIONALES"] > 0,
    df["GANANCIA_(PERDIDA)"] / df["INGRESOS_OPERACIONALES"],
    np.nan,
)

df["roa"] = np.where(
    df["TOTAL_ACTIVOS"] > 0,
    df["GANANCIA_(PERDIDA)"] / df["TOTAL_ACTIVOS"],
    np.nan,
)

df["roe"] = np.where(
    df["TOTAL_PATRIMONIO"] > 0,
    df["GANANCIA_(PERDIDA)"] / df["TOTAL_PATRIMONIO"],
    np.nan,
)

df["leverage"] = np.where(
    df["TOTAL_ACTIVOS"] > 0,
    df["TOTAL_PASIVOS"] / df["TOTAL_ACTIVOS"],
    np.nan,
)

df["ganancia_positiva"] = (df["GANANCIA_(PERDIDA)"] > 0).astype(int)
df["patrimonio_no_positivo"] = (df["TOTAL_PATRIMONIO"] <= 0).astype(int)

df[["margen", "roa", "roe", "leverage"]].describe()


,margen,roa,roe,leverage
count,40000.00000,38863.000000,31304.000000,38863.000000
mean,0.04741,0.034692,0.084589,0.552607
std,0.31900,0.169632,0.427835,0.388691
min,-15.00000,-5.500000,-32.000000,0.000000
25%,0.00000,0.000000,0.000000,0.333333
50%,0.00000,0.000000,0.000000,0.500000
75%,0.00882,0.022472,0.125000,0.771429
max,25.00000,13.500000,18.000000,14.000000


Los descriptores básicos confirman que:

- La mediana de **margen**, **roa** y **roe** tiende a ser cero o muy cercana a cero, reforzando la idea de que la empresa “típica” opera con resultados muy ajustados.
- El **apalancamiento** presenta valores razonables (por ejemplo, alrededor de 0.5 en la mediana), lo que indica una relación relativamente estable entre pasivos y activos.

Sin embargo, los promedios son arrastrados por pocas observaciones extremas, lo que sugiere que la dispersión y los percentiles altos serán más informativos para entender el potencial de rentabilidad sectorial.


### 4.1 Resumen por año y por macrosector

Para entender la dinámica agregada se estudia la evolución de ingresos, ganancias y ratios por año, así como la distribución de rentabilidad por **macro sector**.

Dado que la mediana de la ganancia suele ser cero, se complementa el análisis con percentiles superiores (p75 y p90) y con la proporción de empresas que presentan ganancia positiva.


In [34]:
# Resumen anual (medianas y medias)
summary_year = (
    df.groupby("ANO_DE_CORTE")[
        ["INGRESOS_OPERACIONALES", "GANANCIA_(PERDIDA)", "TOTAL_ACTIVOS",
         "margen", "roa", "roe", "leverage"]
    ]
    .agg(["median", "mean"])
)
summary_year


INGRESOS_OPERACIONALES           GANANCIA_(PERDIDA)            \
                             median      mean             median      mean   
ANO_DE_CORTE                                                                 
2021                           0.03  0.130635                0.0  0.011835   
2022                           0.04  0.172758                0.0  0.016621   
2023                           0.05  0.190872                0.0  0.012566   
2024                           0.05  0.176413                0.0  0.011618   

             TOTAL_ACTIVOS           margen              roa              roe  \
                    median      mean median      mean median      mean median   
ANO_DE_CORTE                                                                    
2021                  0.03  0.180813    0.0  0.049527    0.0  0.035234    0.0   
2022                  0.03  0.215273    0.0  0.056132    0.0  0.039600    0.0   
2023                  0.04  0.230338    0.0  0.041142    0.0  0.031663    0.0   
2024                  0.04  0.245821    0.0  0.042839    0.0  0.032276    0.0   

                       leverage            
                  mean   median      mean  
ANO_DE_CORTE                               
2021          0.086557      0.5  0.532479  
2022          0.101362      0.5  0.566281  
2023          0.077664      0.5  0.561100  
2024          0.073790      0.5  0.550334

La tabla anual muestra que:

- La **mediana de las ganancias** permanece en cero en todos los años observados, aunque las medias son positivas y relativamente estables.
- Los ratios de rentabilidad (medianas de margen, ROA, ROE) también se sitúan en torno a cero, lo que sugiere que la mayor parte del “peso” de las ganancias agregadas proviene de una minoría de empresas con resultados excepcionales.
- El apalancamiento mediano es relativamente estable, lo que refuerza la idea de una estructura financiera agregada sin cambios dramáticos año a año.

Para capturar el “potencial de rentabilidad” de cada macro sector, es más útil estudiar percentiles altos y proporciones de empresas rentables.


In [35]:
def p75(x):
    return x.quantile(0.75)

def p90(x):
    return x.quantile(0.90)

sector_stats = (
    df.groupby(["ANO_DE_CORTE", "MACROSECTOR"])
      .agg(
          med_margen=("margen", "median"),
          p75_margen=("margen", p75),
          p90_margen=("margen", p90),
          med_roa=("roa", "median"),
          p75_roa=("roa", p75),
          p90_roa=("roa", p90),
          med_gan=("GANANCIA_(PERDIDA)", "median"),
          p75_gan=("GANANCIA_(PERDIDA)", p75),
          p90_gan=("GANANCIA_(PERDIDA)", p90),
          prop_margen_pos=("margen", lambda x: (x > 0).mean()),
          prop_gan_pos=("GANANCIA_(PERDIDA)", lambda x: (x > 0).mean()),
      )
      .reset_index()
)

sector_stats.head()


,ANO_DE_CORTE,MACROSECTOR,med_margen,p75_margen,p90_margen,med_roa,p75_roa,p90_roa,med_gan,p75_gan,p90_gan,prop_margen_pos,prop_gan_pos
0,2021,AGROPECUARIO,0.0,0.000000,0.333333,0.0,0.027027,0.172315,0.0,0.00,0.01,0.248918,0.248918
1,2021,COMERCIO,0.0,0.000000,0.062500,0.0,0.000000,0.111111,0.0,0.00,0.01,0.159669,0.159669
2,2021,CONSTRUCCIÓN,0.0,0.000000,0.142857,0.0,0.000000,0.080667,0.0,0.00,0.01,0.169533,0.169533
3,2021,MANUFACTURA,0.0,0.007275,0.125000,0.0,0.013341,0.142857,0.0,0.01,0.02,0.250980,0.250980
4,2021,MINERO,-0.0,0.188244,0.333333,0.0,0.125628,0.333333,0.0,0.01,0.12,0.377551,0.377551


Los resultados sectoriales permiten responder de manera más rica a preguntas como:

- **¿Qué sectores concentran empresas con márgenes altos en el percentil 75 y 90?**
- **¿En qué macro sectores es más frecuente encontrar empresas con ganancias positivas?**

En términos generales, los sectores con mayor `p75_margen`, `p90_margen` y `prop_gan_pos` pueden interpretarse como sectores con mayor **potencial de rentabilidad** dentro de la muestra, mientras que aquellos con valores sistemáticamente bajos reflejan entornos más competitivos o márgenes estructuralmente reducidos.

Este tipo de análisis será un insumo central para el dashboard final, donde se podrán filtrar sectores y años específicos.


## 5. Construcción del panel empresa–año y del target de predicción

El objetivo del modelado es estimar las **ganancias del próximo año** a partir de la información disponible en el año actual. Para ello, se construye un panel emparejando, para cada empresa (`NIT`), las observaciones consecutivas en el tiempo:

- Año t → Año t+1

De esta manera, cada fila del dataset de modelado representa una empresa en el año t, junto con el valor observado de su ganancia en el año t+1.

Adicionalmente, se construye un indicador binario que indica si las ganancias futuras son positivas o no, lo cual habilita un enfoque alternativo de modelado en dos etapas.


In [36]:
df_model = df.copy().sort_values(["NIT", "ANO_DE_CORTE"])

df_model["ANO_NEXT"] = df_model.groupby("NIT")["ANO_DE_CORTE"].shift(-1)
df_model["GANANCIA_T1"] = df_model.groupby("NIT")["GANANCIA_(PERDIDA)"].shift(-1)

mask_pairs = df_model["ANO_NEXT"] == df_model["ANO_DE_CORTE"] + 1
pairs = df_model[mask_pairs].copy()

pairs.rename(columns={"ANO_DE_CORTE": "ANO_T"}, inplace=True)
pairs["ANO_T1"] = pairs["ANO_NEXT"]
pairs["target_gan_t1"] = pairs["GANANCIA_T1"]
pairs["target_bin_t1"] = (pairs["target_gan_t1"] > 0).astype(int)

pairs[["NIT", "ANO_T", "ANO_T1", "GANANCIA_(PERDIDA)", "target_gan_t1", "target_bin_t1"]].head()


,NIT,ANO_T,ANO_T1,GANANCIA_(PERDIDA),target_gan_t1,target_bin_t1
11748,800000118,2021,2022.0,0.01,0.00,0
2155,800000118,2022,2023.0,0.00,0.01,1
10452,800000276,2021,2022.0,0.01,0.02,1
398,800000276,2022,2023.0,0.02,0.01,1
20434,800000276,2023,2024.0,0.01,0.02,1


El panel resultante incluye, para cada empresa y año base `ANO_T`:

- Sus características financieras y de contexto en el año t.
- La ganancia real observada en el año t+1 (`target_gan_t1`).
- Un indicador binario de si la ganancia futura es positiva o no (`target_bin_t1`).

Para respetar la naturaleza temporal del problema, se separa el conjunto de datos en:

- **Entrenamiento**: pares cuyos años base son 2021 y 2022 (para aprender de 2021→2022 y 2022→2023).
- **Prueba**: pares cuya base es 2023 (para evaluar la capacidad de predecir 2023→2024).


In [37]:
# Transformaciones logarítmicas de magnitudes grandes
for col in ["INGRESOS_OPERACIONALES", "GANANCIA_(PERDIDA)",
            "TOTAL_ACTIVOS", "TOTAL_PASIVOS", "TOTAL_PATRIMONIO"]:
    if col == "GANANCIA_(PERDIDA)":
        pos = pairs[col].clip(lower=0)
        pairs["log1p_" + col] = np.log1p(pos)
    else:
        pairs["log1p_" + col] = np.log1p(pairs[col].clip(lower=0))

num_features = [
    "INGRESOS_OPERACIONALES",
    "GANANCIA_(PERDIDA)",
    "TOTAL_ACTIVOS",
    "TOTAL_PASIVOS",
    "TOTAL_PATRIMONIO",
    "margen",
    "roa",
    "roe",
    "leverage",
    "ganancia_positiva",
    "patrimonio_no_positivo",
    "log1p_INGRESOS_OPERACIONALES",
    "log1p_GANANCIA_(PERDIDA)",
    "log1p_TOTAL_ACTIVOS",
    "log1p_TOTAL_PASIVOS",
    "log1p_TOTAL_PATRIMONIO",
]

cat_features = [
    "SUPERVISOR",
    "REGION",
    "DEPARTAMENTO_DOMICILIO",
    "CIUDAD_DOMICILIO",
    "MACROSECTOR",
    "CIIU",
]

time_features = ["ANO_T"]

feature_cols = num_features + cat_features + time_features

# División temporal train / test
train = pairs[pairs["ANO_T"] <= 2022].copy()
test = pairs[pairs["ANO_T"] == 2023].copy()

X_train = train[feature_cols]
X_test = test[feature_cols]

y_train_reg = train["target_gan_t1"]
y_test_reg = test["target_gan_t1"]
y_train_bin = train["target_bin_t1"]
y_test_bin = test["target_bin_t1"]

X_train.shape, X_test.shape


((16762, 23), (8569, 23))

La división de entrenamiento y prueba está basada en el tiempo, lo cual es crucial para evitar **fugas de información**: el modelo sólo ve años pasados en el entrenamiento y se evalúa en el año más reciente (2023→2024).

A continuación se construye un **baseline fuerte** y posteriormente varios modelos más sofisticados para comparar su desempeño.


## 6. Modelos explorados y métrica de evaluación

Dado que las ganancias presentan colas largas (unas pocas empresas muy grandes), se emplean las siguientes métricas de regresión:

- **MAE** (Mean Absolute Error): error medio absoluto, más robusto a outliers.
- **RMSE** (Root Mean Squared Error): penaliza más los errores grandes, relevante cuando las empresas más grandes son las de mayor interés.
- **R²**: proporción de varianza explicada por el modelo.

Como baseline natural se utiliza el modelo de **persistencia**:

> $\hat{y}_{t+1} = y_t$

Es decir, se asume que las ganancias del próximo año son iguales a las del año actual. Este baseline captura gran parte de la dinámica en contextos donde las empresas tienden a mantener niveles similares de resultados a lo largo del tiempo.


In [38]:
def regression_report(y_true, y_pred, name="modelo"):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    print(f"=== {name} ===")
    print(f"MAE : {mae:,.6f}")
    print(f"RMSE: {rmse:,.6f}")
    print(f"R²  : {r2:,.4f}")
    return {"Modelo": name, "MAE": mae, "RMSE": rmse, "R2": r2}

# Baseline: ganancia futura = ganancia actual (año t)
y_pred_base_test = test["GANANCIA_(PERDIDA)"].values
baseline_results = regression_report(y_test_reg, y_pred_base_test, name="Baseline_gan_t - test")
baseline_results

=== Baseline_gan_t - test ===
MAE : 0.011891
RMSE: 0.105345
R²  : 0.7504


{'Modelo': 'Baseline_gan_t - test',
 'MAE': 0.011890535651768001,
 'RMSE': np.float64(0.10534490640575175),
 'R2': 0.7504065560199025}

El baseline de persistencia suele ofrecer un desempeño sorprendentemente bueno en este tipo de problemas, porque muchas empresas operan cerca de un “nivel” estable de ganancias o alrededor de cero.

Por este motivo, el objetivo de los modelos más sofisticados no es simplemente obtener un R² diferente de cero, sino **superar de manera clara** a este baseline en las métricas de interés, especialmente en aquellas que penalizan de forma más severa los errores en empresas grandes (RMSE y R²).


### 6.1 Preprocesamiento para modelos basados en árboles

Se construye un pipeline de preprocesamiento que:

- Imputa valores faltantes en variables numéricas y las escala.
- Imputa valores faltantes en variables categóricas y las convierte a indicadores mediante one-hot encoding.
- Deja el campo de año como una variable numérica adicional.

Aunque los modelos de árboles no requieren escalado de la misma forma que los lineales, incluirlo no perjudica el desempeño y mantiene el pipeline general más homogéneo.


In [39]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features + time_features),
        ("cat", categorical_transformer, cat_features),
    ]
)

### 6.2 Modelo basado en Δganancia (RF_delta)

Una primera aproximación con regresión directa sobre la ganancia mostró señales claras de **sobreajuste**: el modelo explicaba muy bien los datos de entrenamiento pero no superaba consistentemente al baseline en el conjunto de prueba.

Para aprovechar la fortaleza del baseline de persistencia, se adopta una estrategia más refinada:

- En lugar de predecir directamente $\text{ganancia}_{t+1}$, se modela el **cambio** respecto a la ganancia actual:

$\Delta = \text{ganancia}_{t+1} - \text{ganancia}_t$

- El baseline equivale a asumir $\Delta = 0$ para todos los casos.
- Un modelo capaz de predecir variaciones positivas o negativas alrededor de ese baseline puede mejorar especialmente los errores de las empresas más grandes.

A continuación se entrena un **Random Forest Regressor** sobre $\Delta$, usando el pipeline de preprocesamiento definido.


In [40]:
# Construir delta de ganancia
train["delta_gan"] = train["target_gan_t1"] - train["GANANCIA_(PERDIDA)"]
test["delta_gan"] = test["target_gan_t1"] - test["GANANCIA_(PERDIDA)"]

y_train_delta = train["delta_gan"]
y_test_delta = test["delta_gan"]

rf_delta = RandomForestRegressor(
    n_estimators=400,
    max_depth=10,
    min_samples_leaf=20,
    min_samples_split=40,
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

rf_delta_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", rf_delta),
])

rf_delta_pipeline.fit(X_train, y_train_delta)

# Predicción de Δ y reconstrucción de ganancia futura
delta_pred_test = rf_delta_pipeline.predict(X_test)
gan_pred_test = test["GANANCIA_(PERDIDA)"].values + delta_pred_test

rf_delta_results = regression_report(y_test_reg, gan_pred_test, name="RF_delta - test")
rf_delta_results


=== RF_delta - test ===
MAE : 0.012462
RMSE: 0.100244
R²  : 0.7740


{'Modelo': 'RF_delta - test',
 'MAE': 0.012462480746954545,
 'RMSE': np.float64(0.10024358533492059),
 'R2': 0.7739943621257375}

Los resultados de `RF_delta` se comparan directamente con el baseline:

- El **RMSE disminuye** respecto al baseline, indicando que el modelo reduce de forma más eficiente los errores grandes, típicos de las empresas de mayor tamaño.
- El **R² aumenta**, lo que significa que el modelo explica una mayor proporción de la variabilidad de las ganancias futuras.
- El **MAE** puede incrementarse ligeramente, reflejando que el modelo “arriesga” más en ciertos casos, pero el compromiso global es favorable cuando se prioriza el comportamiento en la cola superior de la distribución.

En contextos de política pública o inversión donde las decisiones se concentran en empresas con mayor impacto económico, es razonable priorizar RMSE y R² como métricas principales. A la luz de esto, `RF_delta` se configura como un candidato sólido a modelo final.

### 6.3 Modelo en dos etapas (clasificación + regresión)

Dado que una gran proporción de empresas presenta ganancias futuras exactamente iguales a cero, se exploró también un enfoque de **dos etapas**:

1. Un **clasificador** que estima la probabilidad de que la empresa tenga ganancia positiva en el año t+1.
2. Un **regresor** que estima el tamaño de la ganancia condicional a que ésta sea positiva.

La predicción final se obtiene como el valor esperado:

$\hat{y} = P(\text{ganancia}_{t+1} > 0 \mid X_t)\cdot \hat{y}_{\text{positiva}}$

A continuación se muestra un esquema resumido de este enfoque y su desempeño global.


In [41]:
# Clasificación: probabilidad de ganancia positiva en t+1
rf_clf = RandomForestClassifier(
    n_estimators=400,
    max_depth=10,
    min_samples_leaf=20,
    min_samples_split=40,
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

clf_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", rf_clf),
])

clf_pipeline.fit(X_train, y_train_bin)

proba_test = clf_pipeline.predict_proba(X_test)[:, 1]
pred_test_bin = (proba_test >= 0.5).astype(int)

acc_test = accuracy_score(y_test_bin, pred_test_bin)
auc_test = roc_auc_score(y_test_bin, proba_test)

acc_test, auc_test


(0.8676625043762399, 0.9058872107358256)

El clasificador alcanza una precisión y un AUC elevados, lo que indica una buena capacidad para distinguir entre empresas con ganancias futuras positivas y no positivas.

Sin embargo, la calidad de la predicción final de las ganancias no depende sólo de distinguir el signo, sino también de estimar adecuadamente el tamaño de las ganancias positivas.


In [42]:
# Regresión sólo en casos con ganancia positiva futura
train_pos = train[train["target_gan_t1"] > 0].copy()
test_pos = test[test["target_gan_t1"] > 0].copy()

X_train_pos = train_pos[feature_cols]
y_train_pos = train_pos["target_gan_t1"]
X_test_pos = test_pos[feature_cols]
y_test_pos = test_pos["target_gan_t1"]

rf_reg_pos = RandomForestRegressor(
    n_estimators=400,
    max_depth=10,
    min_samples_leaf=20,
    min_samples_split=40,
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

reg_pos_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", rf_reg_pos),
])

reg_pos_pipeline.fit(X_train_pos, y_train_pos)

y_pred_test_pos = reg_pos_pipeline.predict(X_test_pos)
regression_report(y_test_pos, y_pred_test_pos, name="RegPos - test")


=== RegPos - test ===
MAE : 0.031496
RMSE: 0.285288
R²  : 0.5047


{'Modelo': 'RegPos - test',
 'MAE': 0.031495823870971204,
 'RMSE': np.float64(0.2852876845441332),
 'R2': 0.5046827705255608}

In [43]:
# Combinación: valor esperado para todo el conjunto de prueba
gan_pos_pred_test = reg_pos_pipeline.predict(X_test)
gan_pos_pred_test = np.clip(gan_pos_pred_test, a_min=0, a_max=None)

y_pred_two_stage = proba_test * gan_pos_pred_test

two_stage_results = regression_report(y_test_reg, y_pred_two_stage, name="TwoStage - test")
two_stage_results


=== TwoStage - test ===
MAE : 0.014358
RMSE: 0.161481
R²  : 0.4135


{'Modelo': 'TwoStage - test',
 'MAE': 0.014357726844333445,
 'RMSE': np.float64(0.16148120080783637),
 'R2': 0.4135242883714536}

Aunque la etapa de clasificación funciona muy bien por sí sola, la combinación con la regresión condicional no logra superar al modelo `RF_delta` en términos de RMSE y R². Esto se debe en parte a la dificultad de estimar con precisión el tamaño de las ganancias positivas en presencia de una distribución muy asimétrica.

En conclusión, el enfoque en dos etapas se documenta como una alternativa razonable para este tipo de problema, pero en este dataset concreto no ofrece ventajas claras frente al modelo basado en Δganancia.


### 6.4 Comparación de modelos

Se resumen a continuación las métricas clave de los tres enfoques evaluados en el conjunto de prueba (2023→2024):

- Baseline de persistencia (`Baseline_gan_t`).
- Modelo basado en Δganancia (`RF_delta`).
- Modelo en dos etapas (`TwoStage`).


In [44]:
results_table = pd.DataFrame([baseline_results, rf_delta_results, two_stage_results])
results_table


,Modelo,MAE,RMSE,R2
0,Baseline_gan_t - test,0.011891,0.105345,0.750407
1,RF_delta - test,0.012462,0.100244,0.773994
2,TwoStage - test,0.014358,0.161481,0.413524


Los resultados típicamente muestran un patrón del siguiente tipo:

- El **baseline** ya ofrece un desempeño sólido, especialmente en MAE, al capturar la estabilidad de muchas empresas.
- `RF_delta` logra **mejorar el RMSE y el R²** respecto al baseline, reduciendo los errores grandes en empresas de alto impacto económico.
- El modelo `TwoStage`, pese a ser conceptualmente atractivo para datos con muchos ceros, no logra superar a `RF_delta` en este caso concreto.

En consecuencia, el modelo seleccionado como **modelo final de referencia** es el **Random Forest sobre Δganancia (`RF_delta`)**, que equilibra adecuadamente capacidad explicativa, comportamiento en la cola de la distribución y interpretabilidad.


## 7. Interpretabilidad: importancia de variables

Para entender qué factores impulsan las ganancias futuras en el modelo `RF_delta`, se utiliza **Permutation Importance** sobre el conjunto de prueba. Esta técnica mide cuánto se degrada el desempeño del modelo al permutar aleatoriamente cada característica, una por vez:

- Si la permutación de una variable empeora significativamente el modelo, se considera que esa variable es importante.
- Si no produce cambios apreciables, su contribución marginal es reducida.

Se aplica la técnica directamente sobre el pipeline completo, lo que permite trabajar con los nombres originales de las variables sin preocuparse por los detalles internos del one-hot encoding.


In [45]:
# === (RE)ENTRENAR EL MODELO RF_DELTA PARA ASEGURAR CONSISTENCIA ===
train["delta_gan"] = train["target_gan_t1"] - train["GANANCIA_(PERDIDA)"]
test["delta_gan"] = test["target_gan_t1"] - test["GANANCIA_(PERDIDA)"]

y_train_delta = train["delta_gan"]
y_test_delta = test["delta_gan"]

rf_delta = RandomForestRegressor(
    n_estimators=400,
    max_depth=10,
    min_samples_leaf=20,
    min_samples_split=40,
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

rf_delta_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", rf_delta),
])

rf_delta_pipeline.fit(X_train, y_train_delta)

# === PERMUTATION IMPORTANCE SOBRE EL PIPELINE COMPLETO ===
result = permutation_importance(
    rf_delta_pipeline,
    X_test,
    y_test_reg,   # usamos la ganancia futura como objetivo final
    n_repeats=10,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

importances = pd.DataFrame({
    "feature": feature_cols,
    "importance_mean": result.importances_mean,
    "importance_std": result.importances_std,
}).sort_values("importance_mean", ascending=False)

importances


,feature,importance_mean,importance_std
2,TOTAL_ACTIVOS,2.912556e-02,9.945710e-04
4,TOTAL_PATRIMONIO,2.652426e-02,8.233816e-04
13,log1p_TOTAL_ACTIVOS,2.273901e-02,7.717957e-04
3,TOTAL_PASIVOS,1.512141e-02,1.020356e-03
0,INGRESOS_OPERACIONALES,1.279666e-02,6.370743e-04
14,log1p_TOTAL_PASIVOS,1.168260e-02,8.238508e-04
11,log1p_INGRESOS_OPERACIONALES,6.518168e-03,3.385824e-04
15,log1p_TOTAL_PATRIMONIO,1.726979e-03,1.022796e-04
21,CIIU,9.192882e-04,1.923275e-04
20,MACROSECTOR,5.695754e-04,1.039969e-04


La tabla de importancia de características suele mostrar que:

- Las variables asociadas al **tamaño de la empresa** (`TOTAL_ACTIVOS`, `TOTAL_PATRIMONIO`, `TOTAL_PASIVOS`, `INGRESOS_OPERACIONALES` y sus versiones logarítmicas) son las más influyentes en la predicción de las ganancias futuras.
- Variables categóricas como `CIIU` y `MACROSECTOR` también aportan información, aunque en menor medida, capturando diferencias sectoriales estructurales.
- Otros campos de contexto geográfico (`REGION`, `DEPARTAMENTO_DOMICILIO`, `CIUDAD_DOMICILIO`) tienden a tener importancia marginal frente a los indicadores puramente financieros.

Este patrón es coherente con la intuición económica: las empresas de mayor tamaño y patrimonio tienen un **peso desproporcionado** en la generación de ganancias absolutas, y su historia financiera reciente es un predictor natural de sus resultados futuros.


### 8. Selección del modelo final

Tras evaluar diferentes enfoques de modelado, el modelo seleccionado como referencia para la predicción de ganancias futuras es el **Random Forest sobre el cambio de ganancia (RF_delta)**. Este modelo no intenta predecir directamente la ganancia del año t+1, sino la diferencia entre la ganancia futura y la ganancia actual (Δ = ganancia_{t+1} − ganancia_t). De esta forma, se aprovecha la fortaleza del modelo de persistencia —que asume que la empresa mantendrá el mismo nivel de resultados al año siguiente— y se enfoca el aprendizaje en explicar las desviaciones sistemáticas respecto a ese comportamiento base.

En la comparación empírica sobre el conjunto de prueba (pares 2023→2024), el baseline de persistencia (`Baseline_gan_t`) mostró ya un desempeño sólido, con un **MAE ≈ 0,0119**, **RMSE ≈ 0,1053** y **R² ≈ 0,75**, lo que refleja la estabilidad de los niveles de ganancia para una fracción importante de empresas. Sin embargo, el modelo **RF_delta** logró mejorar de manera consistente estas métricas clave: el **RMSE se redujo a ≈ 0,1002** y el **R² aumentó hasta ≈ 0,77**, aunque el MAE se incrementó ligeramente. Este comportamiento sugiere que RF_delta es especialmente eficaz reduciendo los errores más grandes, precisamente en aquellas empresas de mayor tamaño cuyo impacto económico es más relevante para fines de política pública y decisiones de inversión.

También se exploró un enfoque en dos etapas, combinando un clasificador de ganancia positiva con un regresor condicionado a empresas con ganancia futura mayor que cero. Si bien este esquema obtuvo buenos resultados en términos de clasificación (precisión y AUC elevados), la predicción final de las ganancias no consiguió superar al modelo RF_delta en RMSE ni en R². Por este motivo, se documenta como alternativa razonable pero no se adopta como modelo principal. En síntesis, **RF_delta** ofrece el mejor equilibrio entre capacidad explicativa, control de errores en la cola superior de la distribución y coherencia económica, por lo que se adopta como **modelo final para la proyección de ganancias empresariales** en este trabajo.


## 9. Conclusiones

A partir del proceso completo de limpieza, análisis y modelado se pueden extraer las siguientes conclusiones principales:

1. El dataset de las **10.000 empresas más grandes** presenta una estructura típica de datos empresariales:
   - Muchas empresas con ganancias cercanas a cero.
   - Un subconjunto reducido con ganancias muy grandes que dominan los totales agregados.
   - Fuerte asimetría en las distribuciones financieras y una dinámica temporal relativamente estable.

2. Un **baseline de persistencia** (`ganancia_{t+1} = ganancia_t`) resulta sorprendentemente fuerte, lo que obliga a que cualquier modelo más complejo justifique su uso mediante mejoras concretas en métricas robustas.

3. El enfoque de modelar la **Δganancia** con un **Random Forest** (`RF_delta`) permite:
   - Incorporar información financiera detallada (niveles, razones, transformaciones logarítmicas).
   - Reducir el error cuadrático (RMSE) y aumentar el R² respecto al baseline, especialmente en empresas con grandes volúmenes de activos y patrimonio.

4. El enfoque en **dos etapas** (clasificación + regresión) es conceptualmente adecuado para distribuciones con muchos ceros, pero en este caso no logra superar al modelo `RF_delta` en métricas globales.

5. El análisis de **importancia de variables** confirma el rol central de:
   - El tamaño de la empresa (activos, patrimonio, ingresos).
   - Las diferencias sectoriales (macro sector, CIIU).
   - Mientras que la geografía y otros indicadores de contexto tienen un peso más limitado en este tipo de predicción.